In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
df_claims_keywords = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake.json')

In [7]:
df_claims_keywords.head()

,patent_id,claim_fulltext,keywords_yake
0,10000011,A method of reducing distortion in an additiv...,"[[shrinking supports, 0.0013617439], [part, 0...."
1,10000017,"A method for securing an attachment ( 3 , 33 ...","[[[adhesion surface, 0.0019894983000000002], [..."
2,10000021,A method for manufacturing a three-dimensiona...,"[[[cavity metal mold, 0.0006303933], [predeter..."
3,10000025,A method of laying up plies in an aircraft wi...,"[[[aircraft wing skin, 0.0008585078000000001],..."
4,10000033,"A method of making a storage bag, comprising:...","[[[closure element, 0.002612919], [sidewall, 0..."


In [4]:
df_patent = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/df_patentsview_patent_cpc_grouped_cleantech.json')

In [5]:
df_patent.head()

,patent_id,patent_type,patent_date,patent_title,patent_abstract,wipo_kind,num_claims,withdrawn,filename,cpc
0,10000011,utility,2018-06-19,Supports for sintering additively manufactured...,To reduce distortion in an additively manufact...,B1,30,0,ipg180619.xml,"{'1': {'cpc_sequence': 19, 'cpc_section': 'Y',..."
1,10000017,utility,2018-06-19,Method for mounting a vortex generator and mou...,The invention relates to a method for securing...,B2,21,0,ipg180619.xml,"{'1': {'cpc_sequence': 20, 'cpc_section': 'Y',..."
2,10000021,utility,2018-06-19,Method for manufacturing three-dimensional sha...,There is provided a method for manufacturing a...,B2,4,0,ipg180619.xml,"{'1': {'cpc_sequence': 11, 'cpc_section': 'Y',..."
3,10000025,utility,2018-06-19,Optimized cross-ply orientation in composite l...,A composite laminate has a primary axis of loa...,B2,9,0,ipg180619.xml,"{'1': {'cpc_sequence': 10, 'cpc_section': 'Y',..."
4,10000033,utility,2018-06-19,"Washable, waterproof, sealable and reusable st...",Disclosed are embodiments of a versatile stora...,B2,20,0,ipg180619.xml,"{'1': {'cpc_sequence': 37, 'cpc_section': 'Y',..."


In [8]:
# Merge df_claims_keywords and df_patent, keep columns patent_date, patent_title, cpc from df_patent and all columns from df_claims_keywords
df_patent_claims_keywords = pd.merge(df_claims_keywords, df_patent[['patent_id', 'patent_date', 'patent_title', 'cpc']], on='patent_id', how='left')

In [12]:
df_patent_claims_keywords['cpc'][10]

{'1': {'cpc_sequence': 24,
  'cpc_section': 'Y',
  'cpc_class': 'Y02',
  'cpc_subclass': 'Y02T',
  'cpc_group': 'Y02T10/64',
  'cpc_type': 'additional',
  'cpc_symbol_position': None,
  'cpc_subgroup': 'Y02T10'},
 '2': {'cpc_sequence': 25,
  'cpc_section': 'Y',
  'cpc_class': 'Y02',
  'cpc_subclass': 'Y02T',
  'cpc_group': 'Y02T10/72',
  'cpc_type': 'additional',
  'cpc_symbol_position': None,
  'cpc_subgroup': 'Y02T10'}}

In [24]:
# Extract CPC subclasses and subgroups of patents
def extract_subclass_and_subgroup(cpc_dict):
    subclasses = []
    subgroups = []
    for key, value in cpc_dict.items():
        subclasses.append(value.get('cpc_subclass', None))
        subgroups.append(value.get('cpc_group', None))
    # Only keep unique values
    subclasses = list(set(subclasses))
    subgroups = list(set(subgroups))
    return pd.Series([subclasses, subgroups])

# Apply the function to the 'cpc' column
df_patent_claims_keywords[['cpc_subclass_list', 'cpc_group_list']] = df_patent_claims_keywords['cpc'].progress_apply(extract_subclass_and_subgroup)

100%|██████████| 515742/515742 [00:39<00:00, 13052.07it/s]


In [27]:
df_patent_claims_keywords.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake_cpc.json')